In [4]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

DATA_DIR = Path("../data")
OUT_DIR = Path("../outputs")
OUT_DIR.mkdir(exist_ok=True)
(OUT_DIR / "figures").mkdir(exist_ok=True)

In [5]:
def clean_number(x):
    """Convert '1,234.56' -> 1234.56. Return NaN if not numeric."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip().replace(",", "")
    if s in ["", "None", "nan"]:
        return np.nan
    try:
        return float(s)
    except:
        return np.nan

def normalize_branch(name):
    """Standardize branch names for matching across files."""
    if pd.isna(name) or name is None:
        return None
    s = str(name).lower().strip()
    s = re.sub(r'[^a-z0-9\s]', ' ', s)   # remove punctuation
    s = re.sub(r'\s+', ' ', s).strip()  # collapse spaces
    return s

In [6]:
f4 = DATA_DIR / "rep_s_00673_SMRY.csv"
df4_raw = pd.read_csv(f4)
df4_raw.head(20)

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\rep_s_00673_SMRY.csv'

In [ ]:
df4_raw.columns.tolist()


In [ ]:
df4_raw.head(10)

In [ ]:
df4_raw = df4_raw.rename(columns={
    "Stories": "Category",
    "Unnamed: 1": "Qty",
    "Unnamed: 2": "Total Price",
    "Unnamed: 4": "Total Cost",
    "Unnamed: 5": "Total Cost %",
    "Unnamed: 6": "Total Profit",
    "Unnamed: 8": "Total Profit %"
})

In [ ]:
df4 = df4_raw.copy()

# drop rows where Category is empty
df4 = df4[df4["Category"].notna()].copy()

# remove repeated export header rows
df4 = df4[~df4["Category"].astype(str).str.contains("Theoretical Profit", case=False, na=False)]
df4 = df4[~df4["Category"].astype(str).str.contains("Page", case=False, na=False)]
df4 = df4[df4["Category"].astype(str).str.strip() != "Category"]

df4.head(12)

In [ ]:
df4 = df4_raw.copy()

# keep only rows where Category is not empty
df4 = df4[df4["Category"].notna()].copy()

# remove export junk/header rows
df4 = df4[~df4["Category"].astype(str).str.contains("Theoretical Profit", case=False, na=False)]
df4 = df4[~df4["Category"].astype(str).str.contains("Page", case=False, na=False)]
df4 = df4[~df4["Category"].astype(str).str.match(r"\d{1,2}-[A-Za-z]{3}-\d{2}", na=False)]  # removes 22-Jan-26 style rows
df4 = df4[df4["Category"].astype(str).str.strip() != "Category"]  # removes the row that says Category Qty Total Price...

df4.head(15)


In [ ]:
df4 = df4.reset_index(drop=True)


In [ ]:
# rows that are branch headers start with "Stories"
is_branch_row = df4["Category"].astype(str).str.lower().str.startswith("stories")

# create a helper column that has branch name only on branch rows
df4["branch"] = df4["Category"].where(is_branch_row)

# forward-fill branch downwards
df4["branch"] = df4["branch"].ffill()

df4[["Category","branch"]].head(20)

In [ ]:
df4 = df4[df4["Category"].isin(["BEVERAGES","FOOD"])].copy()
df4.head(10)

In [ ]:
for col in ["Qty", "Total Cost", "Total Profit"]:
    df4[col] = df4[col].apply(clean_number)

df4["Revenue"] = df4["Total Cost"] + df4["Total Profit"]
df4["Profit_Margin"] = df4["Total Profit"] / df4["Revenue"]
df4["branch_norm"] = df4["branch"].apply(normalize_branch)

df4[["branch","Category","Qty","Total Cost","Total Profit","Revenue","Profit_Margin"]].head(10)

In [ ]:
out4 = OUT_DIR / "clean_category_summary.csv"
df4.to_csv(out4, index=False)
print("Saved:", out4)

In [ ]:
df4.shape, df4["branch"].nunique(), df4["Category"].value_counts()

In [ ]:
from io import StringIO

f1 = DATA_DIR / "REP_S_00134_SMRY.csv"

with open(f1, "r", encoding="utf-8", errors="replace") as f:
    text1 = f.read()

df1_raw = pd.read_csv(StringIO(text1), header=None)
df1_raw.head(20)

In [ ]:
df1_raw.shape


In [ ]:
for c in range(min(10, df1_raw.shape[1])):
    sample = df1_raw[c].astype(str)
    if sample.str.contains("Stories", na=False).any():
        print("Found 'Stories' in column:", c)
    

In [ ]:
BRANCH_COL = 1
YEAR_COL = 0

In [ ]:
BRANCH_COL = 1
YEAR_COL = 0

In [ ]:
df1_raw.columns = [f"c{i}" for i in range(df1_raw.shape[1])]

mask_branch = df1_raw[f"c{BRANCH_COL}"].astype(str).str.contains("Stories", na=False)
df1 = df1_raw[mask_branch].copy()

df1.head(10)

In [ ]:
df1["year"] = df1[f"c{YEAR_COL}"].replace("", np.nan).ffill()
df1["year"] = df1["year"].apply(clean_number).astype("Int64")

df1[["year", f"c{BRANCH_COL}"]].head(15)

In [ ]:
import numpy as np
import pandas as pd

# make sure df1 exists and has year + c1 already
df1 = df1.reset_index(drop=True)

months = ["jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]

# Convert everything to numeric where possible (fast)
num_df = df1.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(",", ""), errors="coerce"))

# Extract first 12 numeric values per row
vals = num_df.to_numpy()
out = np.full((vals.shape[0], 12), np.nan)

for i in range(vals.shape[0]):
    rownums = vals[i][~np.isnan(vals[i])]
    # remove year if it appears inside the numeric list
    rownums = rownums[~np.isin(rownums, [2025, 2026])]
    out[i, :min(12, len(rownums))] = rownums[:12]

month_vals = pd.DataFrame(out, columns=months)

df1_fast = pd.concat([df1, month_vals], axis=1)

# quick view
df1_fast[["year", "c1", "jan", "feb", "mar"]].head(10)


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path



In [ ]:
import os
os.getcwd()

In [ ]:
df1_raw = pd.read_csv("../data/REP_S_00134_SMRY.csv", header=None)

In [ ]:
df1_raw = pd.read_csv(r"C:\projects\stories-coffee-analytics\data\REP_S_00134_SMRY.csv", header=None)

In [ ]:
df1_raw.shape


In [ ]:
os.getcwd()

In [ ]:
df1 = df1_raw.copy()

In [ ]:
df1 = df1.apply(lambda col: col.map(clean_number))

In [ ]:
import re
import numpy as np

def clean_number(x):
    if x is None:
        return np.nan
    if isinstance(x, (int, float)):
        return x
    x = str(x).strip()
    if x == "" or x.lower() in ["nan", "none"]:
        return np.nan

    # remove commas and any non-numeric characters (except . and -)
    x = re.sub(r"[^\d\.\-]", "", x)

    try:
        return float(x)
    except:
        return np.nan

In [ ]:
df1 = df1.apply(lambda col: col.map(clean_number))

In [ ]:
df1.head()
df1.dtypes

In [ ]:
df1_raw.head(10)

In [ ]:
import os
from pathlib import Path
print("CWD:", os.getcwd())
print("Repo root exists:", Path("..").resolve())
print("Data files:", list(Path("..").glob("data/*.csv"))[:10])

In [ ]:
import pandas as pd
import numpy as np

df1_raw = pd.read_csv("../data/REP_S_00134_SMRY.csv", header=None)
df1_raw.shape, df1_raw.head(12)

In [ ]:
YEAR_COL = 0
BRANCH_COL = 1
MONTH_START_COL = 3   # January
MONTH_END_COL = 14    # exclusive end (we'll slice 3:15 for 12 months)

In [ ]:
# Find the first row where YEAR_COL looks like 2025/2026 etc
year_numeric = pd.to_numeric(df1_raw[YEAR_COL], errors="coerce")
start_idx = year_numeric.first_valid_index()
start_idx

In [ ]:
df = df1_raw.loc[start_idx:].copy()
df.head(5)

In [ ]:
df["year"] = pd.to_numeric(df[YEAR_COL], errors="coerce").ffill().astype("Int64")
df["branch"] = df[BRANCH_COL].where(
    df[BRANCH_COL].astype(str).str.lower().str.startswith("stories"),
    np.nan
).ffill()

df[["year","branch"]].head(10)

In [ ]:
month_block = df.loc[:, MONTH_START_COL:MONTH_END_COL-1].copy()

# remove commas and convert
month_block = month_block.replace({",": ""}, regex=True)
month_block = month_block.apply(pd.to_numeric, errors="coerce")

month_block.head(5), month_block.dtypes

In [ ]:
month_block = df1.loc[4:, 3:14].copy()
month_block.head()

In [ ]:
month_block.shape

In [ ]:
months = ["jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]

# take the raw month area (from col 3 onward), row 4 onward (your data rows)
raw_month_area = df1.loc[4:, 3:].copy()

def first_12_numbers(row):
    vals = []
    for x in row.values:
        if pd.notna(x):
            vals.append(float(x))
    # pad / trim to exactly 12
    vals = (vals + [np.nan]*12)[:12]
    return pd.Series(vals, index=months)

month_block = raw_month_area.apply(first_12_numbers, axis=1)
month_block.shape, month_block.head()

In [ ]:
month_block.shape

In [ ]:
df = df1.copy()

df["year"] = df[0].ffill()
df["branch"] = df[1].ffill()

# keep only real data rows (where we actually have a branch name)
df_data = df.loc[4:].copy()
df_data = df_data[df_data["branch"].astype(str).str.lower().str.startswith("stories")]
df_data[["year","branch"]].head(10)

In [ ]:
df = df1.copy()
df["year"] = df[0].ffill()
df["branch"] = df[1].ffill()

df.loc[0:25, [0, 1, "year", "branch"]]

In [ ]:
import pandas as pd, numpy as np

# start from raw again (NOT the cleaned df1 that changed types)
df1_raw = pd.read_csv(r"C:\projects\stories-coffee-analytics\data\REP_S_00134_SMRY.csv", header=None)

df = df1_raw.copy()

# year is in col 0, branch name is in col 1 (based on your table)
df["year_raw"] = df[0]
df["branch_raw"] = df[1]

df.loc[0:25, [0, 1, "year_raw", "branch_raw"]]

In [ ]:
df["year"] = pd.to_numeric(df["year_raw"], errors="coerce").ffill()
df["branch"] = df["branch_raw"].astype(str)

# keep only rows where branch starts with "Stories"
mask = df["branch"].str.lower().str.startswith("stories")
df_data = df.loc[mask].copy()

df_data[["year", "branch"]].head(10)

In [ ]:
# find the header row that contains "January"
header_row = df1_raw.apply(lambda r: r.astype(str).str.contains("January", case=False, na=False).any(), axis=1)
header_idx = header_row[header_row].index[0]
header_idx

In [ ]:
df1_raw.loc[header_idx]

In [ ]:
df_data = df_data[df_data["branch"].astype(str).str.lower().str.startswith("stories")]

In [ ]:
df = df1.copy()
df["year"] = df[0].ffill()
df["branch"] = df[1].ffill()

df_tmp = df.loc[4:, ["year","branch"]].reset_index(drop=True)
df_tmp.head(30)

In [ ]:
df_tmp["branch_str"] = df_tmp["branch"].astype(str).str.lower()
df_tmp["has_stories"] = df_tmp["branch_str"].str.contains("stories", na=False)

df_tmp["has_stories"].value_counts(), df_tmp[df_tmp["has_stories"]].head(10)

In [ ]:
df_data = df.loc[4:].reset_index(drop=True).copy()

# normalize branch text
b = df_data["branch"].astype(str).str.lower().str.replace(r"\s+", " ", regex=True).str.strip()

df_data = df_data[b.str.contains(r"\bstories\b", na=False)].reset_index(drop=True)
df_data[["year","branch"]].head(10), df_data.shape

In [ ]:
import pandas as pd
import numpy as np

# 1) Reload raw (IMPORTANT: start from df1_raw, not df1 that got numeric-cleaned)
df1_raw = pd.read_csv(r"..\data\REP_S_00134_SMRY.csv", header=None)

# 2) Find the header row that contains "January"
header_row = df1_raw.apply(lambda r: r.astype(str).str.contains("January", case=False, na=False).any(), axis=1)
header_idx = header_row[header_row].index[0]   # you already saw it's 3

# 3) Keep only rows AFTER header row
raw = df1_raw.loc[header_idx+1:].reset_index(drop=True).copy()

# 4) Build year + branch from ORIGINAL text columns (0 and 1)
raw["year"] = pd.to_numeric(raw[0], errors="coerce").ffill()
raw["branch"] = raw[1].astype(str).replace("nan", np.nan).ffill()

# 5) Month columns are 3..11 in your mapping (Jan..Sep)
month_map = {'jan':3,'feb':4,'mar':5,'apr':6,'may':7,'jun':8,'jul':9,'aug':10,'sep':11}
months = list(month_map.keys())

def clean_number(x):
    if pd.isna(x): 
        return np.nan
    s = str(x).strip().replace(",", "")
    try:
        return float(s)
    except:
        return np.nan

for m, col in month_map.items():
    raw[m] = raw[col].map(clean_number)

# 6) Keep only real branch rows
clean_monthly = raw[raw["branch"].astype(str).str.lower().str.contains("stories", na=False)].copy()

# 7) Total + preview
clean_monthly["total_calc"] = clean_monthly[months].sum(axis=1, skipna=True)

clean_monthly[["year","branch"] + months + ["total_calc"]].head(10), clean_monthly.shape

In [ ]:
months = ["jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]

# add missing months if not present
for m in ["oct","nov","dec"]:
    if m not in clean_monthly.columns:
        clean_monthly[m] = np.nan

# make year an integer type (nullable)
clean_monthly["year"] = pd.to_numeric(clean_monthly["year"], errors="coerce").astype("Int64")

# make sure month columns are numeric
for m in months:
    clean_monthly[m] = pd.to_numeric(clean_monthly[m], errors="coerce")

# recompute total (safe)
clean_monthly["total_calc"] = clean_monthly[months].sum(axis=1, skipna=True)

clean_monthly.head()

In [ ]:
def normalize_branch(s: str) -> str:
    s = str(s)
    s = s.replace("–", "-").replace("—", "-")
    s = " ".join(s.split())  # collapse spaces
    return s.strip()

clean_monthly["branch_norm"] = clean_monthly["branch"].apply(normalize_branch).str.lower()
clean_monthly[["branch","branch_norm"]].head(10)

In [ ]:
clean_monthly.shape
clean_monthly["year"].value_counts(dropna=False).sort_index()
clean_monthly["branch_norm"].nunique(), clean_monthly["branch"].nunique()
clean_monthly.isna().sum().sort_values(ascending=False).head(15)

In [ ]:
from pathlib import Path
OUT_DIR = Path("../outputs")
OUT_DIR.mkdir(exist_ok=True)

out1 = OUT_DIR / "clean_monthly_sales_file1.csv"
clean_monthly.to_csv(out1, index=False)
print("Saved:", out1.resolve())

In [ ]:
months = ["jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]

keep_cols = ["year", "branch", "branch_norm"] + months + ["total_calc"]

clean_monthly = clean_monthly[keep_cols].copy()

clean_monthly.shape, clean_monthly.columns

In [ ]:
clean_monthly.isna().sum().sort_values(ascending=False).head(15)

In [ ]:
out1 = OUT_DIR / "clean_monthly_sales_file1.csv"
clean_monthly.to_csv(out1, index=False)
print("Saved:", out1.resolve())